# Academic Papers RAG System Tutorial

This notebook demonstrates how to build a complete RAG (Retrieval-Augmented Generation) system for academic research papers using LlamaIndex. We'll build it step by step with independent functions that you can run and understand individually.

## What is RAG?

RAG combines the power of:
- **Retrieval**: Finding relevant documents from a knowledge base
- **Augmented Generation**: Using retrieved context to generate informed responses

## System Components

Our RAG system will include:
1. **PDF Processing**: Extract text from academic papers
2. **Document Chunking**: Split documents into searchable segments
3. **Vector Embeddings**: Convert text to numerical representations
4. **Vector Storage**: Store embeddings in LanceDB
5. **Semantic Search**: Find relevant content for queries
6. **Query Engine**: Generate responses using retrieved context


## Understanding StorageContext

This notebook uses LlamaIndex's **StorageContext** approach, which provides significant advantages over simpler vector-only storage methods. Understanding this architecture is crucial for building production-ready RAG systems.

### StorageContext vs. Simple Vector Storage

| Component | StorageContext (This Notebook) | Simple Vector Store | Benefits |
|-----------|-------------------------------|-------------------|----------|
| **Vector Store** | ✅ LanceDB embeddings | ✅ LanceDB embeddings | Fast similarity search |
| **Document Store** | ✅ Original documents preserved | ❌ Lost after processing | Full document access |
| **Index Store** | ✅ Index metadata & structure | ❌ Must rebuild index | Exact reconstruction |
| **Graph Store** | ✅ Document relationships | ❌ No relationship data | Rich context understanding |

### Persistence & Recovery Capabilities

**With StorageContext (Our Approach):**
```python
# Save complete system state
index.storage_context.persist(persist_dir="storage/papers_index")

# Perfect restoration - identical behavior
storage_context = StorageContext.from_defaults(persist_dir="storage/papers_index")
index = load_index_from_storage(storage_context)
# Exact same results every time!
```

**Simple Vector Store Only:**
```python
# Only vectors saved
vector_store = LanceDBVectorStore(uri="./vectors")

# Must recreate everything from scratch
index = VectorStoreIndex.from_vector_store(vector_store)
# May have different behavior, lost metadata
```

### 💡 Key Advantages of StorageContext

1. **Perfect Reproducibility**: Identical results across sessions - critical for research and development
2. **Complete State Management**: All components preserved, not just vectors
3. **Fast Startup**: No reprocessing needed - load existing index instantly
4. **Rich Metadata**: Document relationships, source tracking, and complex queries
5. **Development Friendly**: Iterate without rebuilding entire system
6. **Enterprise Ready**: Robust persistence for production deployments

### 📈 Storage Footprint Example

For 1000 academic papers (~500MB original PDFs):

**StorageContext Storage:**
```
storage/papers_index/
├── docstore.json          # 50MB - Original documents
├── index_store.json       # 5MB  - Index metadata  
├── graph_store.json       # 2MB  - Relationships
└── LanceDB vector files   # 200MB - Embeddings
Total: ~260MB
```

**Benefits**: Complete system restoration, full metadata, relationships preserved

**Simple Vector Storage:**
```
lancedb_data/
└── vectors.lance          # 200MB - Embeddings only
Total: ~200MB
```

**Limitations**: Must rebuild index, lost metadata, no relationships

### 🎯 When to Use StorageContext

- **Research & Development** - Need reproducible experiments  
- **Complex Documents** - Rich metadata and relationships matter  
- **Production Systems** - Robust persistence and recovery required  
- **Academic Work** - Full traceability and citation tracking  
- **Multi-user Systems** - Consistent experience across users  

### 🚀 Performance Impact

- **Initial Build**: ~20% slower (stores additional metadata)
- **Subsequent Loads**: 10x faster (no reprocessing needed)
- **Query Performance**: Identical to simple vector approach
- **Storage Space**: ~30% more storage for complete persistence

This tutorial demonstrates the StorageContext approach because it provides the most robust and feature-complete RAG implementation suitable for real-world applications.


## Mount google drive for the data and files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Environment Setup and Configuration

First, let's set up our environment and load necessary configurations. We'll use OpenRouter for LLM access and local embeddings (no API keys needed for embeddings).


In [ ]:
!pip install -q -r "/content/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 17.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 12.9 MB/s et

In [ ]:
import os
from getpass import getpass

# securely input your key
os.environ["OPENROUTER_API_KEY"] = getpass("Enter your OpenRouter key")
print("✓ OpenRouter key set successfully")

Enter your OpenRouter key··········
✓ OpenRouter key set successfully


In [ ]:
import time
from pathlib import Path
from typing import Dict, List, Optional, Tuple

def setup_environment():
    """
    Setup environment variables and basic configuration.

    Returns:
        bool: Success status
    """

    # Disable tokenizer warning
    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    # Check for required API key
    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        print("OPENROUTER_API_KEY not found in environment variables")
        print("Please add your OpenRouter API key to a .env file")
        return False

    print("✓ Environment variables loaded successfully")
    return True

# Run the setup
success = setup_environment()
if success:
    print("✓ Environment setup complete!")
else:
    print("x Environment setup failed!")

✓ Environment variables loaded successfully
✓ Environment setup complete!


## 2. Configuration Management

Let's define our system configuration directly in the notebook. This includes model settings, chunk sizes, and other parameters.


In [ ]:
# Configuration parameters for the RAG system
CONFIG = {
    "llm": {
        "model": "gpt-4o",                    # OpenRouter model to use
        "temperature": 0.1                   # Temperature for response generation
    },
    "embeddings": {
        "model": "local:BAAI/bge-small-en-v1.5",  # Local embedding model (no API key needed)
        "chunk_size": 512,                 # Size of text chunks for processing
        "chunk_overlap": 50                 # Overlap between consecutive chunks
    },
    "vector_store": {
        "type": "lancedb",                   # Vector database type
        "table_name": "academic_papers",     # Table name for storing embeddings
        "path": "/content/storage/papers_vectordb"    # Path to vector database
    },
    "index": {
        "storage_path": "/content/storage/papers_index",  # Path to store complete index
        "similarity_top_k": 5                             # Number of similar chunks to retrieve
    },
    "papers": {
        "folder": "/content/drive/MyDrive/Outskill RAG/papers/agents"    # Path to academic papers folder
    }
}

def get_config(key_path: str, default_value=None):
    """
    Get configuration value using dot notation.

    Args:
        key_path (str): Dot-separated path to the config value (e.g., 'llm.model')
        default_value: Default value if key not found

    Returns:
        Configuration value or default
    """
    keys = key_path.split('.')
    value = CONFIG

    for key in keys:
        if isinstance(value, dict) and key in value:
            value = value[key]
        else:
            return default_value

    return value

# Test configuration access
llm_model = get_config("llm.model")
embedding_model = get_config("embeddings.model")
chunk_size = get_config("embeddings.chunk_size")

print(f"LLM model: {llm_model}")
print(f"Embedding model: {embedding_model}")
print(f"Chunk size: {chunk_size}")
print("✓ Configuration setup complete")


LLM model: gpt-4o
Embedding model: local:BAAI/bge-small-en-v1.5
Chunk size: 512
✓ Configuration setup complete


## 3. LlamaIndex Settings Configuration

LlamaIndex uses global settings for embeddings, LLMs, and document processing. We'll use OpenRouter for the LLM and a local embedding model (no API key required).


### **a. Settings**

Settings in LlamaIndex is a global configuration object that stores defaults for the entire pipeline like your LLM, embedding model, and node parser.

When you do this:
```
from llama_index.core import Settings
Settings.llm = my_llm
Settings.embed_model = my_embed_model
Settings.node_parser = my_chunker
```

You’re telling LlamaIndex: “Use these components everywhere unless I say otherwise.”

So, when you later create an index or query engine: `index = VectorStoreIndex.from_documents(docs)`, it automatically uses those Settings behind the scenes.

### **b. What does `resolve_embed_model` do?**

Each model type has its own class, parameters, and setup.
The function `resolve_embed_model()` is a shortcut that figures out which embedding class to create based on the name you give it.

You give it a string:

`resolve_embed_model("local:BAAI/bge-small-en-v1.5")` # this is the model string that we configured above

It gives you back a ready-to-use embedding model object, like:
`<SentenceTransformerEmbedding(model_name='BAAI/bge-small-en-v1.5')>`


If you gave it something like: `resolve_embed_model("openai/text-embedding-3-small")`, it would instead return an OpenAIEmbedding object.


With resolve_embed_model(), you just give the name, and it handles everything including downloading the model, caching it, and picking the right wrapper class.

In [ ]:
get_config("llm.temperature")

0.1

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.embeddings import resolve_embed_model
from llama_index.core.node_parser import SentenceSplitter

def configure_llamaindex_settings():
    """
    Configure LlamaIndex global settings for embeddings, LLM, and text processing.
    """
    # Set up LLM with OpenRouter
    model = get_config("llm.model")
    temperature = get_config("llm.temperature", 0.1)

    Settings.llm = OpenRouter(
        api_key=os.getenv("OPENROUTER_API_KEY"),
        model=model,
        temperature=temperature
    )
    print(f"✓ LLM configured: {model} (temperature: {temperature})")

    # Set up local embedding model (downloads first time, then cached)
    embedding_model = get_config("embeddings.model")
    Settings.embed_model = resolve_embed_model(embedding_model)
    print(f"✓ Embedding model configured: {embedding_model}")

    # Set up node parser for chunking
    chunk_size = get_config("embeddings.chunk_size")
    chunk_overlap = get_config("embeddings.chunk_overlap")

    Settings.node_parser = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    print(f"✓ Text chunking configured: {chunk_size} chars with {chunk_overlap} overlap")

# Configure the settings using our hardcoded config
configure_llamaindex_settings()
print("✓ LlamaIndex settings configured successfully")


✓ LLM configured: gpt-4o (temperature: 0.1)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✓ Embedding model configured: local:BAAI/bge-small-en-v1.5
✓ Text chunking configured: 512 chars with 50 overlap
✓ LlamaIndex settings configured successfully


## 4. Vector Store Setup

We'll use LanceDB as our vector database to store document embeddings. LanceDB is a fast, serverless vector database that's perfect for RAG applications.


In [ ]:
from llama_index.vector_stores.lancedb import LanceDBVectorStore

def create_vector_store():
    """
    Create and configure LanceDB vector store using config settings.

    Returns:
        LanceDBVectorStore: Configured vector store
    """
    try:
        import lancedb

        # Get configuration values
        vector_db_path = get_config("vector_store.path")
        table_name = get_config("vector_store.table_name")

        # Basic validation
        if not table_name or not table_name.strip():
            raise ValueError("vector_store.table_name is empty")

        # Only ensure local directories; skip for s3:// or gs://
        # Create storage directory
        if "://" not in vector_db_path:
            Path(vector_db_path).parent.mkdir(parents=True, exist_ok=True)

        # Connect (creates DB dir/files if needed)
        db = lancedb.connect(vector_db_path)
        print(f"✓ Connected to LanceDB at: {vector_db_path}")

        # Create (instantiate) vector store
        vector_store = LanceDBVectorStore(
            uri=str(vector_db_path),
            table_name=table_name
        )
        print(f"✓ LanceDB vector store created (table: {table_name})")

        return vector_store

    except Exception as e:
        print(f"Error creating vector store: {e}")
        return None

# Create the vector store using config
vector_store = create_vector_store()
if vector_store:
    print("✓ Vector store setup complete")
else:
    print("x Vector store setup failed")


✓ Connected to LanceDB at: /content/storage/papers_vectordb
✓ LanceDB vector store created (table: academic_papers)
✓ Vector store setup complete


## 5. PDF Processing and Document Loading

Now we'll create functions to load and process PDF files. We'll use LlamaIndex's native `SimpleDirectoryReader` which can handle PDFs directly without needing a custom processor.

**Internally in `SimpleDirectoryReader`:**
- List files (or source items)
- Select reader for each file based on extension
- Extract text (and maybe images)
- Create metadata (file path, file_name, file_type, sometimes page numbers)
- Return List[Document] ready for parsing/chunking

docs: https://developers.llamaindex.ai/python/framework/module_guides/loading/simpledirectoryreader

In [ ]:
!ls "/content/drive/MyDrive/Outskill RAG/papers/agents"

AI_Agents_vs_Agentic_AI.pdf	       Rise_and_Potential_LLM_Agents.pdf
Emerging_Agent_Architectures.pdf       survey_of_self_evolving_agents.pdf
LLMReasoning_to_Autonomous_Agents.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader

def load_papers_from_folder() -> List:
    """
    Load and process all PDF papers from the configured folder using LlamaIndex's native loader.

    Returns:
        List[Document]: Processed documents ready for indexing
    """
    papers_folder = get_config("papers.folder")
    print(f"Loading papers from: {papers_folder}")

    papers_path = Path(papers_folder)
    if not papers_path.exists():
        print(f"Papers folder does not exist: {papers_path}")
        return []

    # Use LlamaIndex's SimpleDirectoryReader to load PDFs
    # This natively handles PDF parsing, text extraction, and metadata
    documents = SimpleDirectoryReader(papers_folder).load_data()

    print(f"✓ Loaded {len(documents)} documents")
    return documents

# Load the papers using config
documents = load_papers_from_folder()
if documents:
    print(f"Successfully loaded {len(documents)} documents")
    print(f"First document preview: {documents[0].text[:200]}...")
    print(f"First document metadata: {documents[0].metadata}")
else:
    print("No documents loaded")


Loading papers from: /content/drive/MyDrive/Outskill RAG/papers/agents
✓ Loaded 229 documents
Successfully loaded 229 documents
First document preview: AI Agents vs. Agentic AI: A Conceptual
Taxonomy, Applications and Challenges
Ranjan Sapkota ∗‡, Konstantinos I. Roumeliotis †, Manoj Karkee ∗‡
∗Cornell University, Department of Environmental and Biol...
First document metadata: {'page_label': '1', 'file_name': 'AI_Agents_vs_Agentic_AI.pdf', 'file_path': '/content/drive/MyDrive/Outskill RAG/papers/agents/AI_Agents_vs_Agentic_AI.pdf', 'file_type': 'application/pdf', 'file_size': 3196781, 'creation_date': '2025-10-30', 'last_modified_date': '2025-10-30'}


In [ ]:
# Above as you can see 229 documents were created from 5 pdfs, it's because each page in pdf was considered as one document
# If you would like to have the whole pdf together as single document, the below code can be used
# reader = SimpleDirectoryReader(
#     "/path/to/papers",
#     # recursive=True,
#     file_extractor={".pdf": PDFReader(return_full_document=True)}
# )
# documents = reader.load_data()

## 6. Creating the Vector Index

The vector index is the core of our RAG system. It chunks documents, generates embeddings, and stores them in the vector database for efficient similarity search.

---
**“Building an index”** simply means creating a data structure that allows faster or more efficient retrieval of information later.

Whether or not vectors are created depends on what kind of retrieval you want:
- Keyword-based → No vectors (traditional IR)
- Semantic-based → Yes, vectors (modern RAG / AI search)

In RAG systems, when we say “build an index,” we almost always mean creating and storing vector embeddings

### **What happens when you use a node parser**

Internally your Document list is passed to parser.get_nodes_from_documents(documents, ...)

**Under the hood inside the parser:**

- For each document: split text into chunks based on your split strategy
- For each chunk: create a Node object (text + metadata from parent document)
- Optionally: assign relationships such as PREVIOUS/NEXT (sequence), SOURCE (link to the doc), PARENT/CHILD (for hierarchical parser)
- Return List[Node] which will be embedded/indexed

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex, load_index_from_storage

def create_vector_index(documents: List,
                       vector_store,
                       force_rebuild: bool = False):
    """
    Create or load a vector index from documents using config settings.

    Args:
        documents (List): Documents to index
        vector_store: LanceDB vector store
        force_rebuild (bool): Force rebuild even if index exists

    Returns:
        VectorStoreIndex: The created or loaded index
    """
    index_storage_path = get_config("index.storage_path")
    index_path = Path(index_storage_path)
    index_path.mkdir(parents=True, exist_ok=True)

    # Check if index already exists
    index_store_file = index_path / "index_store.json"

    if not force_rebuild and index_store_file.exists():
        print("Loading existing index...")
        try:
            # Recreate storage context with vector store
            storage_context = StorageContext.from_defaults(
                persist_dir=str(index_path),
                vector_store=vector_store
            )

            # Load existing index
            index = load_index_from_storage(storage_context)
            print("✓ Successfully loaded existing index")
            return index

        except Exception as e:
            print(f"Error loading existing index: {e}")
            print("Creating new index...")

    if not documents:
        print("x No documents to index")
        return None

    print("Creating new vector index...")
    start_time = time.time()

    # Create storage context with vector store (configuration layer, this doesn't store anything yet)
    # this tells llamaindex to use LanceDB (in our case) as the storage backend for all vector data
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # Create index with progress bar (here the node parser will be used to create nodes)
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
        show_progress=True
    )

    end_time = time.time()
    print(f"✓ Index created in {end_time - start_time:.2f} seconds")

    # Save index to storage
    print("Saving index to storage...")
    index.storage_context.persist(persist_dir=str(index_path))
    print("✓ Index saved successfully")

    return index

# Create the vector index using config
index = create_vector_index(
    documents=documents,
    vector_store=vector_store,
    force_rebuild=False  # Set to True to force rebuild
)

if index:
    print("✓ Vector index ready for querying")
else:
    print("x Failed to create vector index")


Creating new vector index...


Parsing nodes:   0%|          | 0/229 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/737 [00:00<?, ?it/s]

✓ Index created in 490.65 seconds
Saving index to storage...
✓ Index saved successfully
✓ Vector index ready for querying


In [ ]:
import lancedb

db = lancedb.connect("/content/storage/papers_vectordb")
table = db.open_table("academic_papers")
# check schema
print(table.schema)

row = table.to_pandas().head(1)
print(row)

id: string
doc_id: string
vector: fixed_size_list<item: float>[384]
  child 0, item: float
text: string
metadata: struct<_node_content: string, _node_type: string, creation_date: string, doc_id: string, document_id: string, file_name: string, file_path: string, file_size: int64, file_type: string, last_modified_date: string, page_label: string, ref_doc_id: string>
  child 0, _node_content: string
  child 1, _node_type: string
  child 2, creation_date: string
  child 3, doc_id: string
  child 4, document_id: string
  child 5, file_name: string
  child 6, file_path: string
  child 7, file_size: int64
  child 8, file_type: string
  child 9, last_modified_date: string
  child 10, page_label: string
  child 11, ref_doc_id: string
                                     id                                doc_id  \
0  9fed2c4c-5477-43d3-b0f8-b46a59effe45  3388b3a0-0ed7-4d37-ac45-8b4487d58707   

                                              vector  \
0  [-0.049507886, -0.012100671, 0.008815838, -

## 7. Setting Up the Query Engine

This block builds the Query Engine, which is the layer that combines:

- a Retriever → to find the most relevant chunks from your LanceDB vector store
- an LLM → to synthesize an answer from those chunkse


This is where the "Augmented Generation" part of RAG happens.


In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever

def setup_query_engine(index):
    """
    Setup the query engine for semantic search and response generation using config settings.

    Args:
        index: The vector index to query

    Returns:
        RetrieverQueryEngine: Configured query engine
    """
    if not index:
        print("x Index not available. Please create index first.")
        return None

    try:
        # Get similarity top k from config
        similarity_top_k = get_config("index.similarity_top_k")

        # Create retriever - this finds the most similar document chunks
        retriever = VectorIndexRetriever(
            index=index,
            similarity_top_k=similarity_top_k,
        )
        print(f"✓ Retriever configured to find top {similarity_top_k} similar chunks")

        # Create query engine - this combines retrieval with LLM generation
        query_engine = RetrieverQueryEngine(retriever=retriever)
        print("✓ Query engine setup successfully")

        return query_engine

    except Exception as e:
        print(f"x Error setting up query engine: {e}")
        return None

# Setup the query engine using config
query_engine = setup_query_engine(index)

if query_engine:
    print("✓ RAG system is ready for queries!")
else:
    print("x Failed to setup query engine")


✓ Retriever configured to find top 5 similar chunks
✓ Query engine setup successfully
✓ RAG system is ready for queries!


When you later call `retriever.retrieve(query)`, it:
- Embeds the query using the embedding model defined in Settings.embed_model.
- Performs a vector similarity search against the LanceDB table.
- Returns the top-K most relevant Nodes (chunks) along with their metadata and similarity scores.

At this stage, no LLM is involved yet, it’s just search.

`query_engine = RetrieverQueryEngine(retriever=retriever)`

What it does:
- This wraps the retriever with the LLM you configured in Settings.llm (earlier in your setup).

The RetrieverQueryEngine acts as a pipeline:
- Take the user query (text input).
- Call the retriever to get top-K chunks.
- Combine those chunks (the retrieved context) into a prompt.
- Send that prompt to the LLM.
- Return the generated answer.


        ┌────────────┐
        │ User Query │
        └──────┬─────┘
               │
               ▼
       ┌───────────────┐
       │   Retriever   │─── embed user query, then searches LanceDB for similar chunks
       └───────────────┘
               │
               ▼
       ┌───────────────┐
       │    Context    │
       │   Assembler   │── combines top-K chunks
       └───────────────┘
               │
               ▼
       ┌─────────────────┐
       │ LLM (OpenRouter)│── generates coherent answer
       └─────────────────┘
               │
               ▼
        ┌─────────────┐
        │ Final Answer│
        └─────────────┘


## 8. Search and Query Functions

Now let's create functions to search through our academic papers and extract detailed information about sources and metadata.


In [ ]:
def list_indexed_papers(documents: List) -> List[Dict[str, any]]:
    """
    List all papers that have been indexed with their metadata.

    Args:
        documents (List): List of loaded documents

    Returns:
        List[Dict[str, any]]: List of paper information
    """
    papers = []
    processed_files = set()  # Track unique files to avoid duplicates

    for doc in documents:
        try:
            metadata = doc.metadata
            file_path = metadata.get("file_path", "")
            file_name = Path(file_path).stem if file_path else "Unknown"

            # Skip if we've already processed this file
            if file_path in processed_files:
                continue
            processed_files.add(file_path)

            paper_info = {
                "file_name": file_name,
                "file_path": file_path,
                "file_size": metadata.get("file_size", 0),
                "file_size_mb": round(metadata.get("file_size", 0) / (1024 * 1024), 2) if metadata.get("file_size") else 0,
                "page_label": metadata.get("page_label", ""),
            }

            papers.append(paper_info)

        except Exception as e:
            print(f"Error processing document: {e}")

    return papers

papers_list_improved = list_indexed_papers(documents)

print(f"Found {len(papers_list_improved)} unique papers in the index:\n")

for i, paper in enumerate(papers_list_improved[:5], 1):  # Show first 5 papers
    print("------------"*5)
    print(f"{i}. File: {paper['file_name']}\nSize: {paper['file_size_mb']} MB\nPage label: {paper['page_label']}")


Found 5 unique papers in the index:

------------------------------------------------------------
1. File: AI_Agents_vs_Agentic_AI
Size: 3.05 MB
Page label: 1
------------------------------------------------------------
2. File: Emerging_Agent_Architectures
Size: 1.58 MB
Page label: 1
------------------------------------------------------------
3. File: LLMReasoning_to_Autonomous_Agents
Size: 16.21 MB
Page label: 1
------------------------------------------------------------
4. File: Rise_and_Potential_LLM_Agents
Size: 6.52 MB
Page label: 1
------------------------------------------------------------
5. File: survey_of_self_evolving_agents
Size: 10.56 MB
Page label: 1


In [ ]:
# Search function with metadata extraction
def search_papers(query_engine, query: str, include_metadata: bool = True) -> Dict[str, any]:
    """
    Search for relevant papers based on the query with improved title extraction.

    Args:
        query_engine: The configured query engine
        query (str): Search query
        include_metadata (bool): Whether to include detailed metadata

    Returns:
        Dict[str, any]: Search results with response and sources
    """
    if not query_engine:
        return {
            "success": False,
            "error": "Query engine not initialized.",
            "response": "",
            "sources": [],
        }
    try:
        print(f"Searching for: '{query}'")
        start_time = time.time()

        # Run the RAG query
        response = query_engine.query(query)

        end_time = time.time()

        # Collect retrieved sources (document chunks)
        sources = []
        if hasattr(response, "source_nodes"):
            for node in response.source_nodes:
                node_text = getattr(node, "text", "")

                source_info = {
                    "text": (node_text[:500] + "...") if len(node_text) > 500 else node_text,
                    "score": getattr(node, "score", 0.0),
                }

                # Include document metadata if available
                if include_metadata and hasattr(node, "metadata"):
                    metadata = node.metadata or {}
                    source_info.update({
                        "file_name": metadata.get("file_name", "Unknown"),
                        "file_path": metadata.get("file_path", ""),
                        "page_label": metadata.get("page_label", ""),
                        "file_size_mb": round(metadata.get("file_size", 0) / (1024 * 1024), 2)
                            if metadata.get("file_size") else 0,
                    })

                sources.append(source_info)

        # Build structured result
        result = {
            "success": True,
            "response": str(response),
            "sources": sources,
            "query": query,
            "search_time": round(end_time - start_time, 2),
            "num_sources": len(sources),
        }

        print(f"✓ Search completed in {result['search_time']} seconds")
        print(f"Found {result['num_sources']} relevant sources")

        return result

    except Exception as e:
        print(f"x Error during search: {e}")
        return {"success": False, "error": str(e), "response": "", "sources": []}


In [ ]:
# Test the search with user query
def ask_question(query_engine, question: str, show_sources: bool = True) -> Optional[dict]:
    """
    Ask a question against the RAG system and optionally show brief sources.

    Returns the structured result dict from `search_papers`).
    """
    if not query_engine:
        print("Error: query engine not available.")
        return None

    result = search_papers(query_engine, question, include_metadata=True)
    if not result.get("success"):
        print(f"Error: {result.get('error', 'unknown')}")
        return None

    print(f"Q: {question}\n")
    print("Answer:")
    print(result["response"])
    print(f"\nSearch time: {result['search_time']}s | "
          f"Sources: {result['num_sources']}")

    if show_sources and result["sources"]:
        print("\nSources:")
        for i, src in enumerate(result["sources"], start=1):
            fname = src.get("file_name", "Unknown")
            page = src.get("page_label", "")
            score = src.get("score", 0.0)
            snippet = (src.get("text", "") or "").replace("\n", " ")
            if len(snippet) > 200:
                snippet = snippet[:200] + "..."
            page_info = f" | Page {page}" if page else ""
            print(f"{i}. {fname}{page_info} | score={score:.3f}")
            print(f"   {snippet}")

    return result

# Test with the improved version
test_question = "What are the main architectural patterns for agent systems?"

if query_engine:
    ask_question(query_engine, test_question, show_sources=True)
else:
    print("x Query engine not available")


Searching for: 'What are the main architectural patterns for agent systems?'
✓ Search completed in 3.23 seconds
Found 5 relevant sources
Q: What are the main architectural patterns for agent systems?

Answer:
The main architectural patterns for agent systems include single-agent and multi-agent architectures. Single-agent architectures are typically used for tasks with a narrowly defined list of tools and well-defined processes, making them easier to implement. Multi-agent architectures involve multiple agents that can either use the same or different language models and tools. These are further categorized into vertical and horizontal architectures. Vertical architectures have a lead agent with other agents reporting to it, while horizontal architectures treat all agents as equals, allowing for shared communication and collaboration.

Search time: 3.23s | Sources: 5

Sources:
1. AI_Agents_vs_Agentic_AI.pdf | Page 11 | score=0.721
   As illustrated in Figure 8, the transition begins wi

## 9. Paper Information and Metadata

Let's create functions to list and get detailed information about the papers in our index.


In [ ]:
from typing import List, Dict, Any
from pathlib import Path

def summarize_loaded_documents(documents: List) -> List[Dict[str, Any]]:
    """
    Summarize all documents loaded into LlamaIndex with their file-level metadata.

    This function iterates over each `Document` object returned by the loader
    (e.g., SimpleDirectoryReader) and extracts key information such as file name,
    path, size, and page count. It’s useful for inspecting or verifying which PDFs
    were successfully loaded before building the index.

    Returns one entry per document object (not deduplicated).
    """
    summaries = []

    for doc in documents:
        try:
            metadata = doc.metadata or {}
            file_path = metadata.get("file_path", "")
            file_name = Path(file_path).stem if file_path else "Unknown"

            summaries.append({
                "file_name": file_name,
                "file_path": file_path,
                "page_count": metadata.get("page_count", 0),
                "file_size_mb": round(metadata.get("file_size", 0) / (1024 * 1024), 2)
                    if metadata.get("file_size") else 0,
            })

        except Exception as e:
            print(f"Error processing document: {e}")

    return summaries

# List all indexed papers
papers_list = summarize_loaded_documents(documents)

print(f"------Found {len(papers_list)} papers in the index:------")

for i, paper in enumerate(papers_list, 1):
    print(f"{i}. {paper}")
    print("-" * 40)


------Found 229 papers in the index:------
1. {'file_name': 'AI_Agents_vs_Agentic_AI', 'file_path': '/content/drive/MyDrive/Outskill RAG/papers/agents/AI_Agents_vs_Agentic_AI.pdf', 'page_count': 0, 'file_size_mb': 3.05}
----------------------------------------
2. {'file_name': 'AI_Agents_vs_Agentic_AI', 'file_path': '/content/drive/MyDrive/Outskill RAG/papers/agents/AI_Agents_vs_Agentic_AI.pdf', 'page_count': 0, 'file_size_mb': 3.05}
----------------------------------------
3. {'file_name': 'AI_Agents_vs_Agentic_AI', 'file_path': '/content/drive/MyDrive/Outskill RAG/papers/agents/AI_Agents_vs_Agentic_AI.pdf', 'page_count': 0, 'file_size_mb': 3.05}
----------------------------------------
4. {'file_name': 'AI_Agents_vs_Agentic_AI', 'file_path': '/content/drive/MyDrive/Outskill RAG/papers/agents/AI_Agents_vs_Agentic_AI.pdf', 'page_count': 0, 'file_size_mb': 3.05}
----------------------------------------
5. {'file_name': 'AI_Agents_vs_Agentic_AI', 'file_path': '/content/drive/MyDrive/Outs

## 11. Advanced Query Examples

Now let's test our RAG system with various types of research queries to demonstrate its capabilities.


In [ ]:
def run_example_queries(query_engine):
    """
    Run a series of example queries to demonstrate RAG capabilities.

    Args:
        query_engine: The configured query engine
    """
    example_queries = [
        ("Agent Types", "What are the main types of AI agents discussed in these papers?"),
        ("Technical Comparison", "How do LLM-based agents differ from traditional AI agents?"),
        ("Challenges", "What are the current challenges in developing autonomous agents?"),
        ("Evaluation", "What evaluation methods are used for AI agents?"),
        ("Architecture", "Describe the common architectural patterns for agent systems."),
        ("Applications", "What are the practical applications of AI agents mentioned in the literature?"),
    ]

    print("Running Example Queries")
    print("=" * 60)

    for i, (category, query) in enumerate(example_queries, start=1):
        print(f"\n{i}. {category}")
        print(f"Q: {query}")
        print("-" * 50)

        result = search_papers(query_engine, query)
        if not result.get("success"):
            print(f"Error: {result.get('error', 'Unknown error')}")
            continue

        # Display concise response
        response = result["response"]
        if len(response) > 400:
            response = response[:400].rstrip() + "..."

        print(f"A: {response}")
        print(f"Sources: {result['num_sources']} | Time: {result['search_time']}s")

    print("\nAll example queries completed.")

# Run the example queries
if query_engine:
    run_example_queries(query_engine)
else:
    print("x Query engine not available for examples")


Running Example Queries

1. Agent Types
Q: What are the main types of AI agents discussed in these papers?
--------------------------------------------------
Searching for: 'What are the main types of AI agents discussed in these papers?'
✓ Search completed in 1.71 seconds
Found 5 relevant sources
A: The main types of AI agents discussed include AI Agents and Agentic AI. AI Agents are autonomous software entities designed for goal-directed task execution within digital environments, characterized by their ability to perceive inputs, reason over contextual information, and initiate actions to achieve specific objectives. Agentic AI, on the other hand, is associated with more complex goals and i...
Sources: 5 | Time: 1.71s

2. Technical Comparison
Q: How do LLM-based agents differ from traditional AI agents?
--------------------------------------------------
Searching for: 'How do LLM-based agents differ from traditional AI agents?'
✓ Search completed in 1.22 seconds
Found 5 relevant sou

## 12. Interactive Query Interface

Let's create an interactive function that allows you to ask custom questions about the papers.


In [ ]:
def ask_question(query_engine, question: str, show_sources: bool = True):
    """
    Ask a custom question to the RAG system and display results.

    Args:
        query_engine: The configured query engine
        question (str): Your question about the papers
        show_sources (bool): Whether to display source information
    """
    print(f"Question: {question}")

    result = search_papers(query_engine, question, include_metadata=True)

    if result["success"]:
        print(f"Answer: {result['response']}")
        print(f"\nSearch completed in {result['search_time']:.2f} seconds")
        print(f"Found {result['num_sources']} relevant sources")

        if show_sources and result["sources"]:
            print(f"\nSource Details:")
            print("-" * 40)
            for i, source in enumerate(result["sources"], 1):
                print(f"\n{i}. Score: {source.get('score', 0):.3f}")
                print(f"   Text: {source['text'][:200]}...")

    else:
        print(f"x Error: {result['error']}")

# Example usage - you can modify this question
custom_question = "What are the key ethical considerations for AI agents?"

if query_engine:
    ask_question(query_engine, custom_question, show_sources=True)
else:
    print("x Query engine not available")


Question: What are the key ethical considerations for AI agents?
Searching for: 'What are the key ethical considerations for AI agents?'
✓ Search completed in 1.38 seconds
Found 5 relevant sources
Answer: Key ethical considerations for AI agents include ensuring they adhere to moral and ethical guidelines that align with human societal values. They should provide accurate and truthful information, express uncertainty when unable to provide answers, and maintain a stance of harmlessness by avoiding biases, discrimination, and dangerous actions. Additionally, AI agents should be capable of adapting to specific demographics and contexts, exhibiting appropriate social values. Ethical governance frameworks are essential to ensure responsible deployment, defining accountability, oversight, and value alignment across autonomous agent networks.

Search completed in 1.38 seconds
Found 5 relevant sources

Source Details:
----------------------------------------

1. Score: 0.669
   Text: Conseque

## Conclusion

🎉 **Congratulations!** You have successfully built a complete RAG (Retrieval-Augmented Generation) system for academic papers using LlamaIndex.

### What we accomplished:

1. **Environment Setup**: Configured API keys and dependencies
2. **Configuration Management**: Loaded system settings from YAML files
3. **LlamaIndex Configuration**: Set up embeddings, LLM, and text processing
4. **Vector Store**: Created a LanceDB vector database for storing embeddings
5. **Document Processing**: Loaded and processed PDF academic papers
6. **Vector Indexing**: Created searchable vector embeddings of documents
7. **Query Engine**: Set up retrieval and response generation
8. **Search Functions**: Implemented semantic search with metadata
9. **Paper Analysis**: Created functions for listing and summarizing papers
10. **Interactive Queries**: Built an interface for asking custom questions
11. **Performance Analytics**: Added system statistics and monitoring

### Key Features:

- **Semantic Search**: Find relevant content using natural language queries
- **Source Attribution**: Get detailed citations and references for answers
- **Metadata Integration**: Access file path, file name, page number, etc.
- **Performance Monitoring**: Track search times and system statistics
- **Flexible Configuration**: Easy to modify models, chunk sizes, and parameters

### Next Steps:

1. **Experiment** with different queries to explore your document collection
2. **Modify** the `custom_question` variable to ask your own questions
3. **Adjust** parameters like `chunk_size`, `similarity_top_k` for different results
4. **Add** more papers to the `papers/agents` folder and rebuild the index
5. **Enhance** the system with additional features like filtering or ranking

### Usage Tips:

- Use specific, focused questions for better results
- Try different phrasings of the same question
- Check the source information to understand where answers come from
- Experiment with the `similarity_top_k` parameter to get more or fewer sources

Happy researching! 🔬📚
